###PROGRAMAÇÃO COM PYSPARK.
###Avaliação Final.
#####Vamos trabalhar com 2 datasets que têm dados sobre a temperatura (em Farenheit) de uma cidade.

In [0]:
#Paths dos datasets
display(dbutils.fs.ls('dbfs:/databricks-datasets/weather/'))

path,name,size,modificationTime
dbfs:/databricks-datasets/weather/README.weather_history.md,README.weather_history.md,750,1540403318000
dbfs:/databricks-datasets/weather/high_temps,high_temps,19180,1540403358000
dbfs:/databricks-datasets/weather/low_temps,low_temps,19180,1540403342000


####1. Leia e visualize os dois datasets (dica: ambos estão em formato csv)

In [0]:
higth_temps = spark.read. format("csv")\
                        . option("header", "true")\
                        . option("inferSchema" , "true")\
                        . load("/databricks-datasets/weather/high_temps")

display(higth_temps.limit(10))

date,temp
2015-01-01T00:00:00.000+0000,42
2015-01-02T00:00:00.000+0000,42
2015-01-03T00:00:00.000+0000,41
2015-01-04T00:00:00.000+0000,51
2015-01-05T00:00:00.000+0000,54
2015-01-06T00:00:00.000+0000,54
2015-01-07T00:00:00.000+0000,46
2015-01-08T00:00:00.000+0000,46
2015-01-09T00:00:00.000+0000,50
2015-01-10T00:00:00.000+0000,46


In [0]:
low_temps = spark.read. format("csv")\
                        . option("header", "true")\
                        . option("inferSchema" , "true")\
                        . load("/databricks-datasets/weather/low_temps" )

display(low_temps.limit(10))

date,temp
2015-01-01T00:00:00.000+0000,26
2015-01-02T00:00:00.000+0000,32
2015-01-03T00:00:00.000+0000,35
2015-01-04T00:00:00.000+0000,38
2015-01-05T00:00:00.000+0000,49
2015-01-06T00:00:00.000+0000,43
2015-01-07T00:00:00.000+0000,42
2015-01-08T00:00:00.000+0000,35
2015-01-09T00:00:00.000+0000,38
2015-01-10T00:00:00.000+0000,43


####2. Conte o número de linhas de ambos.

In [0]:
higth_temps.count()

Out[5]: 1369

In [0]:
low_temps.count()

Out[6]: 1369

####3. Verifique se há valores repetidos na coluna "date" de ambos os datasets.

In [0]:
import pyspark.sql.functions as F

display(higth_temps.select(F.countDistinct('date')))
display(low_temps.select(F.countDistinct('date')))

count(DISTINCT date)
1369


count(DISTINCT date)
1369


####4. Verifique se há valores nulos nos datasets.

In [0]:
higth_temps.filter('date is Null').count()


Out[8]: 0

In [0]:
low_temps.filter('date is Null').count()

Out[9]: 0

####5. Verifique as datas mínima e máxima de cada dataset.

In [0]:
higth_temps.agg(F.min("date"),
                F.max("date")).show()

+-------------------+-------------------+
|          min(date)|          max(date)|
+-------------------+-------------------+
|2015-01-01 00:00:00|2018-09-30 00:00:00|
+-------------------+-------------------+



In [0]:
low_temps.agg(F.min("date"),
              F.max("date")).show()

+-------------------+-------------------+
|          min(date)|          max(date)|
+-------------------+-------------------+
|2015-01-01 00:00:00|2018-09-30 00:00:00|
+-------------------+-------------------+



####6. Verifique o schema dos datasets.

In [0]:
higth_temps.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- temp: integer (nullable = true)



In [0]:
low_temps.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- temp: integer (nullable = true)



####7. Faça um JOIN das duas tabelas, de forma que tenhamos apenas um dataset com as colunas "date", "higher_temp" e "lower_temp".

In [0]:
higth_temps = higth_temps.withColumnRenamed('temp','higher_temp')
low_temps = low_temps.withColumnRenamed('temp', 'lower_temp')

In [0]:
df_temps = higth_temps.join(low_temps, on=['date'], how='inner')

display(df_temps.limit(10))

date,higher_temp,lower_temp
2015-01-01T00:00:00.000+0000,42,26
2015-01-02T00:00:00.000+0000,42,32
2015-01-03T00:00:00.000+0000,41,35
2015-01-04T00:00:00.000+0000,51,38
2015-01-05T00:00:00.000+0000,54,49
2015-01-06T00:00:00.000+0000,54,43
2015-01-07T00:00:00.000+0000,46,42
2015-01-08T00:00:00.000+0000,46,35
2015-01-09T00:00:00.000+0000,50,38
2015-01-10T00:00:00.000+0000,46,43


####8. Verifique o número de linhas da nova tabela e se há algum valor nulo (dica: o número de linhas deve ser idêntico ao número de linhas das tabelas separadas)

In [0]:
df_temps.count()

Out[17]: 1369

####9. Crie uma coluna chamada "amplitude", que será higher_temp - lower_temp

In [0]:
df_temps = df_temps.withColumn('amplitude',
                             df_temps.higher_temp - df_temps.lower_temp)

display(df_temps.limit(10))

date,higher_temp,lower_temp,amplitude
2015-01-01T00:00:00.000+0000,42,26,16
2015-01-02T00:00:00.000+0000,42,32,10
2015-01-03T00:00:00.000+0000,41,35,6
2015-01-04T00:00:00.000+0000,51,38,13
2015-01-05T00:00:00.000+0000,54,49,5
2015-01-06T00:00:00.000+0000,54,43,11
2015-01-07T00:00:00.000+0000,46,42,4
2015-01-08T00:00:00.000+0000,46,35,11
2015-01-09T00:00:00.000+0000,50,38,12
2015-01-10T00:00:00.000+0000,46,43,3


####10. Encontre os dias com a maior e a menor amplitude de temperatura.

In [0]:
max_ampl_temp = df_temps.select('date', 'amplitude')\
                        .sort(df_temps.amplitude.desc())
        
display(max_ampl_temp.limit(10))

date,amplitude
2016-04-18T00:00:00.000+0000,35
2018-07-29T00:00:00.000+0000,34
2016-05-13T00:00:00.000+0000,34
2018-05-13T00:00:00.000+0000,33
2016-05-02T00:00:00.000+0000,33
2018-08-09T00:00:00.000+0000,33
2015-07-04T00:00:00.000+0000,33
2015-08-01T00:00:00.000+0000,32
2017-06-25T00:00:00.000+0000,32
2018-07-26T00:00:00.000+0000,32


In [0]:
min_ampl_temp = df_temps.select('date', 'amplitude')\
                        .sort('amplitude')
        
display(min_ampl_temp.limit(10)) 

date,amplitude
2017-11-16T00:00:00.000+0000,2
2015-12-13T00:00:00.000+0000,3
2015-01-10T00:00:00.000+0000,3
2017-02-18T00:00:00.000+0000,3
2017-12-02T00:00:00.000+0000,3
2016-01-04T00:00:00.000+0000,3
2016-01-23T00:00:00.000+0000,3
2016-12-11T00:00:00.000+0000,3
2018-01-12T00:00:00.000+0000,3
2018-02-01T00:00:00.000+0000,3


####11. Crie uma coluna chamada "mean", que será a média da temperatura máxima e mínima de cada dia.

In [0]:
df_temps = df_temps.withColumn('mean',
                    (df_temps.higher_temp + df_temps.lower_temp) / 2)

display(df_temps.limit(10))

date,higher_temp,lower_temp,amplitude,mean
2015-01-01T00:00:00.000+0000,42,26,16,34.0
2015-01-02T00:00:00.000+0000,42,32,10,37.0
2015-01-03T00:00:00.000+0000,41,35,6,38.0
2015-01-04T00:00:00.000+0000,51,38,13,44.5
2015-01-05T00:00:00.000+0000,54,49,5,51.5
2015-01-06T00:00:00.000+0000,54,43,11,48.5
2015-01-07T00:00:00.000+0000,46,42,4,44.0
2015-01-08T00:00:00.000+0000,46,35,11,40.5
2015-01-09T00:00:00.000+0000,50,38,12,44.0
2015-01-10T00:00:00.000+0000,46,43,3,44.5


####12. Encontre os dias com a maior e a menor média de temperatura.

In [0]:
max_mean_temp = df_temps.select('date', 'mean')\
                        .sort(df_temps.mean.desc())

display(max_mean_temp.limit(10))

date,mean
2016-08-19T00:00:00.000+0000,82.0
2017-08-03T00:00:00.000+0000,81.0
2017-06-25T00:00:00.000+0000,80.0
2017-08-02T00:00:00.000+0000,80.0
2015-07-19T00:00:00.000+0000,79.0
2015-07-31T00:00:00.000+0000,79.0
2016-06-05T00:00:00.000+0000,78.5
2015-07-30T00:00:00.000+0000,78.5
2015-07-02T00:00:00.000+0000,78.5
2015-07-03T00:00:00.000+0000,78.0


In [0]:
min_mean_temp = df_temps.select('date', 'mean')\
                        .sort('mean')

display(min_mean_temp.limit(10))

date,mean
2017-01-03T00:00:00.000+0000,27.0
2017-01-05T00:00:00.000+0000,28.0
2016-12-17T00:00:00.000+0000,28.0
2017-01-04T00:00:00.000+0000,29.0
2016-12-16T00:00:00.000+0000,29.5
2017-01-06T00:00:00.000+0000,30.0
2018-02-21T00:00:00.000+0000,30.0
2017-01-02T00:00:00.000+0000,30.0
2017-01-11T00:00:00.000+0000,31.0
2017-01-13T00:00:00.000+0000,31.0


####13. Encontre uma solução para particionarmos o dataset por ANO e MÊS. (dica: "yyyy-MM")

In [0]:
df_temp_csv = df_temps.withColumn("year", F.date_format("date", 'yyyy'))\
                      .withColumn("month", F.date_format("date", 'MM'))

display(df_temp_csv.limit(10))

date,higher_temp,lower_temp,amplitude,mean,year,month
2015-01-01T00:00:00.000+0000,42,26,16,34.0,2015,01
2015-01-02T00:00:00.000+0000,42,32,10,37.0,2015,01
2015-01-03T00:00:00.000+0000,41,35,6,38.0,2015,01
2015-01-04T00:00:00.000+0000,51,38,13,44.5,2015,01
2015-01-05T00:00:00.000+0000,54,49,5,51.5,2015,01
2015-01-06T00:00:00.000+0000,54,43,11,48.5,2015,01
2015-01-07T00:00:00.000+0000,46,42,4,44.0,2015,01
2015-01-08T00:00:00.000+0000,46,35,11,40.5,2015,01
2015-01-09T00:00:00.000+0000,50,38,12,44.0,2015,01
2015-01-10T00:00:00.000+0000,46,43,3,44.5,2015,01


####14. Salve o novo dataset em uma pasta de sua preferência, particionado por ANO e MÊS.

In [0]:
df_temp_csv.write.option("header",True) \
           .partitionBy("year","month") \
           .mode("overwrite") \
           .csv("/growdev/new_dfTemps.csv")

In [0]:
display(dbutils.fs.ls('/growdev/new_dfTemps.csv'))

####15. Agrupe o dataset por ANO e MÊS e encontre os seguintes valores agregados:
* Temperatura máxima de cada mês
* Temperatura mínima de cada mês
* Temperatura média de cada mês
* Amplitude de cada mês

In [0]:
new_dfTemps = spark.read.format("csv")\
                 . option("header", "true")\
                 . option("inferSchema" , "true")\
                 . load('/growdev/new_dfTemps.csv')

display(new_dfTemps.limit(10))

date,higher_temp,lower_temp,amplitude,mean,year,month
2018-05-01T00:00:00.000+0000,62,48,14,55.0,2018,5
2018-05-02T00:00:00.000+0000,70,45,25,57.5,2018,5
2018-05-03T00:00:00.000+0000,74,50,24,62.0,2018,5
2018-05-04T00:00:00.000+0000,64,50,14,57.0,2018,5
2018-05-05T00:00:00.000+0000,68,48,20,58.0,2018,5
2018-05-06T00:00:00.000+0000,77,52,25,64.5,2018,5
2018-05-07T00:00:00.000+0000,69,53,16,61.0,2018,5
2018-05-08T00:00:00.000+0000,75,50,25,62.5,2018,5
2018-05-09T00:00:00.000+0000,67,52,15,59.5,2018,5
2018-05-10T00:00:00.000+0000,62,50,12,56.0,2018,5


In [0]:

df_weather_history = new_dfTemps.groupBy("year", "month") \
                                .agg(F.max('higher_temp').alias('higher_temp'),\
                                     F.min('lower_temp').alias('lower_temp'),
                                     F.mean('mean').alias('mean'),
                                     F.mean('amplitude').alias('amplitude'))\
                                .sort('year', 'month')
        
display(df_weather_history.limit(10))

year,month,higher_temp,lower_temp,mean,amplitude
2015,1,63,26,45.064516129032256,10.451612903225806
2015,2,62,33,48.75,11.571428571428571
2015,3,69,31,50.516129032258064,14.709677419354838
2015,4,77,37,51.36666666666667,17.066666666666666
2015,5,82,43,59.12903225806452,17.806451612903224
2015,6,92,49,67.66666666666667,22.466666666666665
2015,7,95,54,71.24193548387096,22.677419354838708
2015,8,92,54,68.70967741935483,20.516129032258064
2015,9,81,45,60.5,16.066666666666666
2015,10,74,45,57.24193548387097,12.67741935483871
